In [ ]:
import pandas as pd
import nltk
import random
import re
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize 
nltk.download('punkt')
import matplotlib.pyplot as plt
import nltk
import PyPDF2
from wordcloud import WordCloud
from PyPDF2 import PdfReader
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from tabulate import tabulate

# BNP Paribas SA (More of an Integrated Report, Not Annaul Report)

In [ ]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text

In [ ]:
pdf_path = r"C:\Users\gabri\Documents\BNP Paribas SA.pdf"
text_to_scrape = extract_text_from_pdf(pdf_path)

1 Word

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (1,1)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-5:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

2 Words

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (2,2)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-5:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

3 Words

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (3,3)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-5:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

# HSBC Holdings PLC

In [ ]:
pdf_path = r"C:\Users\gabri\Documents\HSBC Holdings PLC.pdf"
text_to_scrape = extract_text_from_pdf(pdf_path)

1 Word

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (1,1)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

2 Words

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (2,2)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

3 Words

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (3,3)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

# HSBC Holdings PLC (Customer Service)

In [ ]:
pdf_path = r"C:\Users\gabri\Documents\HSBC Holdings PLC (customer experience).pdf"
text_to_scrape = extract_text_from_pdf(pdf_path)

1 Word

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (1,1)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

2 Words

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (2,2)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

3 Words

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (3,3)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

# Crédit Agricole Group

In [ ]:
pdf_path = r"C:\Users\gabri\Documents\Crédit Agricole Group.pdf"
text_to_scrape = extract_text_from_pdf(pdf_path)

1 Word

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (1,1)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

2 Words

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (2,2)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

3 Words

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (3,3)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

# Barclays PLC

In [ ]:
pdf_path = r"C:\Users\gabri\Documents\Barclays PLC.pdf"
text_to_scrape = extract_text_from_pdf(pdf_path)

1 Word

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (1,1)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

2 Words

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (2,2)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

3 Words

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (3,3)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

# Banco Santander SA

In [ ]:
pdf_path = r"C:\Users\gabri\Documents\Banco Santander SA.pdf"
text_to_scrape = extract_text_from_pdf(pdf_path)

1 Word

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (1,1)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

2 Words

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (2,2)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

3 Words

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (3,3)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

# Groupe BPCE

This was More of a Result Page, not an Annual Report. Could not seem to find a Dedicated Annual Report, thus this bank was not analyzed in that capacity

# Société Générale SA (Integrated Report, Not Annual Report)

In [ ]:
pdf_path = r"C:\Users\gabri\Documents\Société Générale SA.pdf"
text_to_scrape = extract_text_from_pdf(pdf_path)

1 Words

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (1,1)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

2 Words

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (2,2)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

3 Words

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (3,3)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

# Deutsche Bank AG

In [ ]:
pdf_path = r"C:\Users\gabri\Documents\Deutsche Bank.pdf"
text_to_scrape = extract_text_from_pdf(pdf_path)

1 Word

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (1,1)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

2 Words

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (2,2)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

3 Words

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation and lowercasing
    tokens = [word.lower() for word in tokens if word.isalpha()]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Preprocess the documents
preprocessed_documents = [preprocess_text(text_to_scrape)]

number_of_topic_words = (3,3)

# Create a pipeline with CountVectorizer and LatentDirichletAllocation
num_topics = 1
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=number_of_topic_words)),
    ('lda', LatentDirichletAllocation(n_components=num_topics, random_state=42)),
])

# Fit the pipeline on the preprocessed data
pipeline.fit(preprocessed_documents)

# Get the top words for each topic
feature_names = pipeline.named_steps['vect'].get_feature_names_out()
lda_components = pipeline.named_steps['lda'].components_
top_words_per_topic = []

for topic_weights in lda_components:
    top_word_indices = topic_weights.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_words_per_topic.append(top_words)

vectorizer = CountVectorizer(ngram_range=number_of_topic_words)  # Change ngram_range as needed
X = vectorizer.fit_transform(preprocessed_documents)

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

top_word_counts = {}

for top_words in top_words_per_topic:
    for word in top_words:
        if word in feature_names:
            word_index = list(feature_names).index(word)  # Convert feature_names to list and find index
            word_frequency = X[:, word_index].sum()
            top_word_counts[word] = word_frequency

# Sort the top words by frequency
sorted_top_words = sorted(top_word_counts.items(), key=lambda x: x[1], reverse=True)

# Extract top words and their frequencies
top_words = [word for word, count in sorted_top_words]
word_frequencies = [count for word, count in sorted_top_words]

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_words)), word_frequencies, color='skyblue')
plt.yticks(range(len(top_words)), labels=top_words)
plt.xlabel('Frequency')
plt.ylabel('Top Words')
plt.title('Frequency of Top Words')
plt.gca().invert_yaxis()  # Invert y-axis to have the most frequent word on top
plt.show()

# Create a table
table_data = [(word, freq) for word, freq in zip(top_words, word_frequencies)]

# Print the table
print(tabulate(table_data, headers=['Top Words', 'Frequency']))

# Combine top words and their frequencies into a dictionary
word_freq_dict = {word: freq for word, freq in zip(top_words, word_frequencies)}

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Words')
plt.show()

# Intesa Sanpaolo SpA